# Weather information: Top 35 cities to visit in France  

In this notebook I :  
-  Retrieved the weather information for the [top 35 best cities in France](https://one-week-in.com/35-cities-to-visit-in-france/)
-  Defined the top 5 cities to visit based on the best weather for the next 7 days

## Import libraries

In [1]:
import requests
import pandas as pd
import numpy as np
import json

from src.cities_weather import req_gps_fn, req_weather_fn, req_to_df_fn
from src.cities_weather import explode_df_fn, explode2_df_fn, explodeall_df_fn

## Import data

In [2]:
# Import list of top 35 cities in France
filename = 'data/raw/top_35_cities_france.txt'
with open(filename) as file:
    top_cities_ls = [line.rstrip() for line in file]

print(len(top_cities_ls))
print(top_cities_ls[0:4])

35
['Mont Saint Michel', 'Saint-Malo', 'Bayeux', 'Le Havre']


## 1. GPS coordinates

Use the [nominatim API](https://nominatim.org/) to get latitute and longitude for each of the cities of interest 


### API request

In [3]:
# Use previously defined requests function  to retrieve response for top cities
api_gps=[req_gps_fn(x) for x in top_cities_ls]
print('API response length: ', len(api_gps))
api_gps[0]

API response length:  35


[{'place_id': 151486647,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'way',
  'osm_id': 211285890,
  'boundingbox': ['48.6349172', '48.637031', '-1.5133292', '-1.5094796'],
  'lat': '48.6359541',
  'lon': '-1.511459954959514',
  'display_name': 'Mont Saint-Michel, Le Mont-Saint-Michel, Avranches, Manche, Normandie, France métropolitaine, 50170, France',
  'place_rank': 20,
  'category': 'place',
  'type': 'islet',
  'importance': 0.755436556781574},
 {'place_id': 282955240,
  'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
  'osm_type': 'relation',
  'osm_id': 7360493,
  'boundingbox': ['46.7308335', '46.9105744', '-75.4379111', '-75.1592812'],
  'lat': '46.7798558',
  'lon': '-75.336261',
  'display_name': 'Mont-Saint-Michel, Antoine-Labelle, Laurentides, Québec, J0W 1P0, Canada',
  'place_rank': 16,
  'category': 'boundary',
  'type': 'administrative',
  'importance': 0.7301414688716277,
  '

In [4]:
# Create dataframe from request response
api_cities_df = req_to_df_fn(api_gps)
api_cities_df.head()

Number of API keys retrieved in request response:  12
List of dataframes per API result. Length:  35
Merged API requests dataframe shape:  (294, 12)


,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,place_rank,category,type,importance
0,151486647,"Data © OpenStreetMap contributors, ODbL 1.0. h...",way,211285890,"[48.6349172, 48.637031, -1.5133292, -1.5094796]",48.6359541,-1.511459954959514,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",20,place,islet,0.755437
1,282955240,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,7360493,"[46.7308335, 46.9105744, -75.4379111, -75.1592...",46.7798558,-75.336261,"Mont-Saint-Michel, Antoine-Labelle, Laurentide...",16,boundary,administrative,0.730141
2,65638299,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,5972710523,"[48.6359711, 48.6360711, -1.5115447, -1.5114447]",48.6360211,-1.5114947,"Mont Saint-Michel, Le Mont-Saint-Michel, Avran...",18,natural,peak,0.600000
3,14994654,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,1581196576,"[43.8495943, 43.8496943, 7.0938427, 7.0939427]",43.8496443,7.0938927,"Mont Saint-Michel, Les Ferres, Grasse, Alpes-M...",18,natural,peak,0.600000
4,334476680,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,9581659906,"[43.897659, 43.897759, 7.3643673, 7.3644673]",43.897709,7.3644173,"Mont Saint-Michel, Nice, Alpes-Maritimes, Prov...",18,natural,peak,0.600000


In [5]:
# Keep copy of original df
api_cities_df_raw=api_cities_df
api_cities_df=api_cities_df_raw.copy()

### Delete duplicated alias

In [6]:
#Create a column containing search name of city (obtained from display_name column)
api_cities_df['Search_Name']=api_cities_df.display_name.str.split(',',expand=True)[0]
api_cities_df.head()

#Sort by entry with highest importance (given based on Wikipedia entries) and then delete duplicated Search Names
api_cities_df=api_cities_df.sort_values('importance', ascending=False).drop_duplicates(subset=['Search_Name'])
api_cities_df=api_cities_df.sort_values('importance', ascending=False).drop_duplicates(subset=['Search_Name'])
print('Shape of duplicate clean cities dataframe: ', api_cities_df.shape)
api_cities_df.head()

Shape of duplicate clean cities dataframe:  (72, 13)


,place_id,licence,osm_type,osm_id,boundingbox,lat,lon,display_name,place_rank,category,type,importance,Search_Name
0,282321559,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,414459,"[43.4062445, 43.5858463, 4.2301364, 4.6444785]",43.4522771,4.4287172,"Saintes-Maries-de-la-Mer, Arles, Bouches-du-Rh...",16,boundary,administrative,0.972517,Saintes-Maries-de-la-Mer
1,283556983,"Data © OpenStreetMap contributors, ODbL 1.0. h...",node,17807753,"[48.6934951, 49.0134951, 2.1883915, 2.5083915]",48.8534951,2.3483915,"Paris, Île-de-France, France métropolitaine, 7...",15,place,city,0.941710,Paris
0,281716908,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,70279,"[43.4461058, 43.6259224, 5.2694745, 5.5063013]",43.5298424,5.4474738,"Aix-en-Provence, Bouches-du-Rhône, Provence-Al...",16,boundary,administrative,0.891085,Aix-en-Provence
0,282341149,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,104492,"[49.4516697, 49.5401463, 0.0667992, 0.1955556]",49.4938975,0.1079732,"Le Havre, Seine-Maritime, Normandie, France mé...",16,boundary,administrative,0.822333,Le Havre
0,281822562,"Data © OpenStreetMap contributors, ODbL 1.0. h...",relation,117858,"[46.1331804, 46.1908971, -1.2419231, -1.111097]",46.1591126,-1.1520434,"La Rochelle, Charente-Maritime, Nouvelle-Aquit...",16,boundary,administrative,0.801484,La Rochelle


### GPS coordinates dataframe

In [7]:
#The GPS coordinates are found under the keys lat and lon
cols_keep_ls=["Search_Name","lat","lon"]
GPS_df=api_cities_df.loc[:,cols_keep_ls]

print('Cities GPS dataframe shape: ', GPS_df.shape)
GPS_df.head(44)

Cities GPS dataframe shape:  (72, 3)


,Search_Name,lat,lon
0,Saintes-Maries-de-la-Mer,43.4522771,4.4287172
1,Paris,48.8534951,2.3483915
0,Aix-en-Provence,43.5298424,5.4474738
0,Le Havre,49.4938975,0.1079732
0,La Rochelle,46.1591126,-1.1520434
0,Lyon,45.7578137,4.8320114
0,Marseille,43.2961743,5.3699525
0,Strasbourg,48.584614,7.7507127
1,Toulouse,43.6007049,1.4241145487393414
0,Bormes-les-Mimosas,43.1572172,6.329253867921363


In [8]:
#When filtering df by  names from initial list of cities we loose some of them. Update name in top cities file and re-do API request
s=pd.Series(top_cities_ls)
print(s[~s.isin(GPS_df["Search_Name"])])

Series([], dtype: object)


In [9]:
# Final GPS info dataframe. Select search names in list of top cities
GPS_df=GPS_df[GPS_df["Search_Name"].isin(top_cities_ls)]
GPS_df.sort_values(by=['Search_Name'],inplace=True)
print(GPS_df.shape)
GPS_df.head()

(35, 3)


,Search_Name,lat,lon
1,Aigues Mortes,45.3314132,1.03014
0,Aix-en-Provence,43.5298424,5.4474738
1,Amiens,49.898700500000004,2.2724569947750464
0,Annecy,45.8992348,6.1288847
0,Ariège,42.9455368,1.4065544156065486


## 2. Weather data  
Use the [openweathermap API](https://openweathermap.org/api/one-call-api) and previously obtained GPS coordinates to retrieve weather information for the cities of interest

In [10]:
# Define key needed for use API (obtained after registration)
API_key = open("access_keys/API_Key_OpenWeather.txt", "r").read()

In [11]:
#Latitude and longitude form df as lists
lat=GPS_df["lat"].values.tolist()
lon=GPS_df["lon"].values.tolist()

#List comprehension to get request for all of the cities found (by latitude and longitude)
api_weath=[req_weather_fn(x,y, key=API_key) for x,y in zip(lat,lon)]

print('API weather results length: ', len(api_weath)) 
print('API weather keys length: ',len(api_weath[0])) 
api_weath[0]

API weather results length:  35
API weather keys length:  6


{'lat': 45.3314,
 'lon': 1.0301,
 'timezone': 'Europe/Paris',
 'timezone_offset': 7200,
 'current': {'dt': 1650304023,
  'sunrise': 1650258335,
  'sunset': 1650307469,
  'temp': 17.5,
  'feels_like': 16.9,
  'pressure': 1014,
  'humidity': 61,
  'dew_point': 9.9,
  'uvi': 0.14,
  'clouds': 97,
  'visibility': 10000,
  'wind_speed': 3.45,
  'wind_deg': 304,
  'wind_gust': 7.43,
  'weather': [{'id': 804,
    'main': 'Clouds',
    'description': 'overcast clouds',
    'icon': '04d'}]},
 'daily': [{'dt': 1650279600,
   'sunrise': 1650258335,
   'sunset': 1650307469,
   'moonrise': 1650316380,
   'moonset': 1650261660,
   'moon_phase': 0.56,
   'temp': {'day': 18.65,
    'min': 7.33,
    'max': 20.18,
    'night': 14.92,
    'eve': 17.67,
    'morn': 7.33},
   'feels_like': {'day': 17.85, 'night': 14.35, 'eve': 17.03, 'morn': 7.33},
   'pressure': 1016,
   'humidity': 49,
   'dew_point': 7.42,
   'wind_speed': 3.45,
   'wind_deg': 304,
   'wind_gust': 7.43,
   'weather': [{'id': 803,
     '

In [12]:
# Create dataframe from request response
weather_df_raw = req_to_df_fn(api_weath)
weather_df_raw.head()

Number of API keys retrieved in request response:  6
List of dataframes per API result. Length:  35
Merged API requests dataframe shape:  (35, 6)


,lat,lon,timezone,timezone_offset,current,daily
0,45.3314,1.0301,Europe/Paris,7200,"{'dt': 1650304023, 'sunrise': 1650258335, 'sun...","[{'dt': 1650279600, 'sunrise': 1650258335, 'su..."
0,43.5298,5.4475,Europe/Paris,7200,"{'dt': 1650304023, 'sunrise': 1650257451, 'sun...","[{'dt': 1650279600, 'sunrise': 1650257451, 'su..."
0,49.8987,2.2725,Europe/Paris,7200,"{'dt': 1650304023, 'sunrise': 1650257536, 'sun...","[{'dt': 1650279600, 'sunrise': 1650257536, 'su..."
0,45.8992,6.1289,Europe/Paris,7200,"{'dt': 1650304023, 'sunrise': 1650257055, 'sun...","[{'dt': 1650279600, 'sunrise': 1650257055, 'su..."
0,42.9455,1.4066,Europe/Paris,7200,"{'dt': 1650304023, 'sunrise': 1650258474, 'sun...","[{'dt': 1650279600, 'sunrise': 1650258474, 'su..."


In [13]:
#Drop current weather information. Keep only daily
weather_df=weather_df_raw.copy().reset_index(drop=True)
weather_df.drop('current',axis=1,inplace=True)
print(weather_df.shape)

(35, 5)


In [15]:
# First data explosion: on chosen column: daily
df_explode = explode_df_fn(weather_df, 'daily')
print(df_explode.shape)
df_explode.head()

(35, 12)


,lat,lon,timezone,timezone_offset,Today,Day_1,Day_2,Day_3,Day_4,Day_5,Day_6,Day_7
0,45.3314,1.0301,Europe/Paris,7200,"{'dt': 1650279600, 'sunrise': 1650258335, 'sun...","{'dt': 1650366000, 'sunrise': 1650344632, 'sun...","{'dt': 1650452400, 'sunrise': 1650430930, 'sun...","{'dt': 1650538800, 'sunrise': 1650517229, 'sun...","{'dt': 1650625200, 'sunrise': 1650603529, 'sun...","{'dt': 1650711600, 'sunrise': 1650689830, 'sun...","{'dt': 1650798000, 'sunrise': 1650776132, 'sun...","{'dt': 1650884400, 'sunrise': 1650862435, 'sun..."
1,43.5298,5.4475,Europe/Paris,7200,"{'dt': 1650279600, 'sunrise': 1650257451, 'sun...","{'dt': 1650366000, 'sunrise': 1650343753, 'sun...","{'dt': 1650452400, 'sunrise': 1650430057, 'sun...","{'dt': 1650538800, 'sunrise': 1650516362, 'sun...","{'dt': 1650625200, 'sunrise': 1650602667, 'sun...","{'dt': 1650711600, 'sunrise': 1650688974, 'sun...","{'dt': 1650798000, 'sunrise': 1650775281, 'sun...","{'dt': 1650884400, 'sunrise': 1650861590, 'sun..."
2,49.8987,2.2725,Europe/Paris,7200,"{'dt': 1650279600, 'sunrise': 1650257536, 'sun...","{'dt': 1650366000, 'sunrise': 1650343816, 'sun...","{'dt': 1650452400, 'sunrise': 1650430098, 'sun...","{'dt': 1650538800, 'sunrise': 1650516380, 'sun...","{'dt': 1650625200, 'sunrise': 1650602663, 'sun...","{'dt': 1650711600, 'sunrise': 1650688948, 'sun...","{'dt': 1650798000, 'sunrise': 1650775233, 'sun...","{'dt': 1650884400, 'sunrise': 1650861520, 'sun..."
3,45.8992,6.1289,Europe/Paris,7200,"{'dt': 1650279600, 'sunrise': 1650257055, 'sun...","{'dt': 1650366000, 'sunrise': 1650343351, 'sun...","{'dt': 1650452400, 'sunrise': 1650429647, 'sun...","{'dt': 1650538800, 'sunrise': 1650515944, 'sun...","{'dt': 1650625200, 'sunrise': 1650602242, 'sun...","{'dt': 1650711600, 'sunrise': 1650688541, 'sun...","{'dt': 1650798000, 'sunrise': 1650774841, 'sun...","{'dt': 1650884400, 'sunrise': 1650861142, 'sun..."
4,42.9455,1.4066,Europe/Paris,7200,"{'dt': 1650279600, 'sunrise': 1650258474, 'sun...","{'dt': 1650366000, 'sunrise': 1650344778, 'sun...","{'dt': 1650452400, 'sunrise': 1650431084, 'sun...","{'dt': 1650538800, 'sunrise': 1650517390, 'sun...","{'dt': 1650625200, 'sunrise': 1650603697, 'sun...","{'dt': 1650711600, 'sunrise': 1650690005, 'sun...","{'dt': 1650798000, 'sunrise': 1650776315, 'sun...","{'dt': 1650884400, 'sunrise': 1650862625, 'sun..."


In [16]:
# Second data explosion: explode weather info for every day
names_7days=[str(df_explode.columns[x]) for x in range(5,12)]
td=["Today"]
days_ls=td+names_7days

weather_perday_df = explodeall_df_fn(days_ls, df_exploded=df_explode)
weather_perday_df.head(44)

Explosion done for all: 8 days
Shape of df:(280, 33)
Verify unique days: ['Today' 'Day_1' 'Day_2' 'Day_3' 'Day_4' 'Day_5' 'Day_6' 'Day_7']


,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,moonrise,moonset,moon_phase,...,temp_night,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,rain,Day,snow
0,45.3314,1.0301,Europe/Paris,7200,1650279600,1650258335,1650307469,1650316380,1650261660,0.56,...,14.92,17.67,7.33,17.85,14.35,17.03,7.33,NaN,Today,NaN
1,43.5298,5.4475,Europe/Paris,7200,1650279600,1650257451,1650306234,1650314880,1650260820,0.56,...,14.58,16.40,10.87,16.79,13.71,15.51,9.61,NaN,Today,NaN
2,49.8987,2.2725,Europe/Paris,7200,1650279600,1650257536,1650307673,1650317100,1650260580,0.56,...,10.72,13.15,6.66,15.18,10.12,12.71,5.26,NaN,Today,NaN
3,45.8992,6.1289,Europe/Paris,7200,1650279600,1650257055,1650306302,1650315180,1650260340,0.56,...,13.04,18.34,3.98,15.31,11.89,17.25,3.98,NaN,Today,NaN
4,42.9455,1.4066,Europe/Paris,7200,1650279600,1650258474,1650307150,1650315840,1650261900,0.56,...,8.53,9.91,4.65,10.36,8.53,9.45,4.65,NaN,Today,NaN
5,43.9492,4.8059,Europe/Paris,7200,1650279600,1650257565,1650306428,1650315120,1650260940,0.56,...,15.96,20.24,9.53,18.71,15.20,19.55,9.08,NaN,Today,NaN
6,49.2765,-0.7025,Europe/Paris,7200,1650283200,1650258323,1650308313,1650317700,1650261420,0.56,...,8.64,11.14,8.23,12.27,7.23,10.37,6.70,NaN,Today,NaN
7,43.4933,-1.4751,Europe/Paris,7200,1650283200,1650259114,1650307893,1650316680,1650262500,0.56,...,13.18,14.82,12.50,14.65,12.85,14.50,12.23,0.72,Today,NaN
8,-33.9659,25.508,Africa/Johannesburg,7200,1650276000,1650256951,1650297106,1650301560,1650262860,0.56,...,15.43,15.91,17.24,19.51,15.30,15.75,17.19,0.82,Today,NaN
9,43.4711,-1.5527,Europe/Paris,7200,1650283200,1650259134,1650307910,1650316680,1650262560,0.56,...,12.89,14.09,12.39,13.94,12.56,13.75,12.09,0.68,Today,NaN


In [17]:
# Export weather df
weather_perday_df.to_csv('data/processed/top_35_cities_france_weather_7days.csv', index=False)

## 3. Final dataset: Expanded weather data for 7 days for 35 cities

City Name included with lat and lon  
1 row equivalent to 1 day weather info for a specific city

In [18]:
# Merge GPS_df to weather_perday_df to get name of city

# Before merging: update type of columns in GPS df
GPS_df['lat']=pd.to_numeric(GPS_df["lat"])
GPS_df['lon']=pd.to_numeric(GPS_df["lon"])

# Round lat and lon to signif figures for merging
GPS_rounded_df=GPS_df.copy()
GPS_rounded_df['lat']=[np.round(num,4) for num in GPS_rounded_df['lat']]
GPS_rounded_df['lon']=[np.round(num,4) for num in GPS_rounded_df['lon']]

GPS_rounded_df.to_csv('data/processed/top_35_cities_france_gpscoord.csv', index=False)

GPS_rounded_df.head()

,Search_Name,lat,lon
1,Aigues Mortes,45.3314,1.0301
0,Aix-en-Provence,43.5298,5.4475
1,Amiens,49.8987,2.2725
0,Annecy,45.8992,6.1289
0,Ariège,42.9455,1.4066


In [19]:
# Final, merged dataframe. Lost info for Collioure despite updating on latitude and longitude 
gps_7dweather_df=weather_perday_df.merge(GPS_rounded_df,left_on=["lat","lon"],right_on=["lat","lon"])
print(gps_7dweather_df.shape)
print('{} unique cities: {}'.format(gps_7dweather_df.Search_Name.nunique(), gps_7dweather_df.Search_Name.unique()))
gps_7dweather_df.head()

(272, 34)
34 unique cities: ['Aigues Mortes' 'Aix-en-Provence' 'Amiens' 'Annecy' 'Ariège' 'Avignon'
 'Bayeux' 'Bayonne' 'Besancon' 'Biarritz' 'Bormes-les-Mimosas'
 'Carcassonne' 'Cassis' 'Château du Haut-Kœnigsbourg' 'Colmar' 'Dijon'
 'Eguisheim' 'Gorges du Verdon' 'Grenoble' 'La Rochelle' 'Le Havre'
 'Lille' 'Lyon' 'Marseille' 'Mont Saint Michel' 'Montauban' 'Nîmes'
 'Paris' 'Rouen' 'Saint-Malo' 'Saintes-Maries-de-la-Mer' 'Strasbourg'
 'Toulouse' 'Uzès']


,lat,lon,timezone,timezone_offset,dt,sunrise,sunset,moonrise,moonset,moon_phase,...,temp_eve,temp_morn,feels_like_day,feels_like_night,feels_like_eve,feels_like_morn,rain,Day,snow,Search_Name
0,45.3314,1.0301,Europe/Paris,7200,1650279600,1650258335,1650307469,1650316380,1650261660,0.56,...,17.67,7.33,17.85,14.35,17.03,7.33,NaN,Today,NaN,Aigues Mortes
1,45.3314,1.0301,Europe/Paris,7200,1650366000,1650344632,1650393947,0,1650349920,0.60,...,11.63,9.06,13.78,7.92,11.09,8.51,9.62,Day_1,NaN,Aigues Mortes
2,45.3314,1.0301,Europe/Paris,7200,1650452400,1650430930,1650480424,1650407640,1650438840,0.64,...,9.95,8.97,9.90,7.07,9.77,8.32,16.99,Day_2,NaN,Aigues Mortes
3,45.3314,1.0301,Europe/Paris,7200,1650538800,1650517229,1650566901,1650498660,1650528360,0.67,...,11.14,7.80,10.62,9.00,10.53,7.80,3.86,Day_3,NaN,Aigues Mortes
4,45.3314,1.0301,Europe/Paris,7200,1650625200,1650603529,1650653378,1650588960,1650618600,0.71,...,14.03,6.59,14.66,10.48,13.42,5.83,0.14,Day_4,NaN,Aigues Mortes


In [20]:
# Export final dataframe with weather for 7 days for top french cities to visit
gps_7dweather_df.to_csv('results/top_35_cities_france_weather_gps.csv', index=False)